In [2]:
import mindspore as ms
import mindspore.ops as ops

x = ops.arange(4.0)
x

Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  1.00000000e+00,  2.00000000e+00,  3.00000000e+00])

在我们$y$计算关于$\\{x}$的梯度之前，我们需要介绍一下MindSpore的自动微分实现方式

MindSpore现有版本不像Pytorch一样将梯度grad直接绑定在Tensor上，而是整体运算后，再通过获取梯度的算子进行梯度的提取。因此，和Pytorch有如下差异：

   1.想要自动微分的函数需要显式注册为function

   2.需要通过自动微分接口`mindspore.grad`或者`mindspore.value_and_grad`来获取梯度

现在计算$y$

In [3]:
import mindspore.numpy as mnp
from mindspore import grad

def forward(x):
    return 2 * mnp.dot(x, x)

y = forward(x)
y

Tensor(shape=[], dtype=Float32, value= 28)

通过调用`mindspore.grad`算子来自动计算y关于x每个分量的梯度

In [8]:
x_grad = grad(forward)(x)
x_grad

Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  4.00000000e+00,  8.00000000e+00,  1.20000000e+01])

In [9]:
x_grad == 4 * x

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

现在让我们计算x的另一个函数

In [11]:
def forward(x):
    return x.sum()

x_grad = grad(forward)(x)
x_grad

Tensor(shape=[4], dtype=Float32, value= [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00])

In [12]:
def forward(x):
    y = x * x
    return y.sum()

x_grad = grad(forward)(x)
x_grad

Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  2.00000000e+00,  4.00000000e+00,  6.00000000e+00])

In [13]:
def forward(x):
    y = x * x
    u = ops.stop_gradient(y)
    z = u * x
    return z, u

z, u = forward(x)
x_grad = grad(forward)(x)
x_grad == u

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

In [14]:
def forward(x):
    y = x * x
    return y.sum()
x_grad = grad(forward)(x)
x_grad == 2 * x

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

In [15]:
def f(a):
    b = a * 2
    while ops.norm(b, dim=0) < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [19]:
a = ops.randn(())
d = f(a)
a_grad = grad(f)(a)

In [20]:
a_grad == d / a

Tensor(shape=[], dtype=Bool, value= True)